In [1]:
import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
print(net.fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [3]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

这里模型结构是为了适配ImageNet预训练的权重，因此最后全连接层（fc）的输出节点数是1000。

In [3]:
input_tensor = torch.randn(32,3,224,224)
net(input_tensor).shape

torch.Size([32, 1000])

## 1、修改模型若干层

假设我们要用这个`resnet`模型去做一个10分类的问题，就应该修改模型的`fc`层，将其输出节点数替换为10。另外，我们觉得一层全连接层可能太少了，想再加一层。可以做如下修改：

In [4]:
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 128)),
                          ('relu1', nn.ReLU()), 
                          ('dropout1',nn.Dropout(0.5)),
                          ('fc2', nn.Linear(128, 10)),
                          ('output', nn.Softmax(dim=1))
                          ]))
net.fc = classifier
print(net.fc)

Sequential(
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (output): Softmax(dim=1)
)


这里的操作相当于将模型`net`最后名称为`fc`的层替换成了名称为`classifier`的结构，该结构是我们自己定义的。这里使用了`Sequential+OrderedDict`的模型定义方式。至此，我们就完成了模型的修改，现在的模型就可以去做10分类任务了。

In [5]:
input_tensor = torch.randn(32,3,224,224)
net(input_tensor).shape

torch.Size([32, 10])

## 2、添加额外输入

有时候在模型训练中，除了已有模型的输入之外，还需要输入额外的信息。比如在CNN网络中，我们除了输入图像，还需要同时输入图像对应的其他信息，这时候就需要在已有的CNN网络中添加额外的输入变量。基本思路是：将原模型添加输入位置前的部分作为一个整体，同时在`forward`中定义好原模型不变的部分、添加的输入和后续层之间的连接关系，从而完成模型的修改。

我们以`torchvision`的`resnet50`模型为基础，任务还是10分类任务。不同点在于，我们希望利用已有的模型结构，在倒数第二层增加一个额外的输入变量`add_variable`来辅助预测。具体实现如下：

In [6]:
class Model(nn.Module):
    def __init__(self, net):
        super(Model, self).__init__()
        self.net = net
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc_add = nn.Linear(1001, 10, bias=True)
        # dim=1是按行还是按列？
        self.output = nn.Softmax(dim=1)
        
    def forward(self, x, add_variable):
        x = self.net(x)
        x = torch.cat((self.dropout(self.relu(x)), add_variable.unsqueeze(1)),1)
        x = self.fc_add(x)
        x = self.output(x)
        return x

这里的实现要点是通过`torch.cat`实现了`tensor`的拼接。`torchvision`中的`resnet50`输出是一个1000维的`tensor`，我们通过修改forward函数（配套定义一些层），先将1000维的tensor通过激活函数层和dropout层，再和外部输入变量`add_variable`拼接，最后通过全连接层映射到指定的输出维度10。

In [7]:
net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model = Model(net)
add_variable = torch.randn(32)
model(input_tensor, add_variable).shape

torch.Size([32, 10])

## 3、添加额外输出

有时候在模型训练中，除了模型最后的输出外，我们需要输出模型某一中间层的结果，以施加额外的监督，获得更好的中间层结果。基本的思路是修改模型定义中`forward`函数的`return`变量。

我们依然以`resnet50`做10分类任务为例，在已经定义好的模型结构上，同时输出1000维的倒数第二层和10维的最后一层结果。具体实现如下：

In [8]:
class Model(nn.Module):
    def __init__(self, net):
        super(Model, self).__init__()
        self.net = net
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(1000, 10, bias=True)
        self.output = nn.Softmax(dim=1)
        
    def forward(self, x):
        x1000 = self.net(x)
        x10 = self.dropout(self.relu(x1000))
        x10 = self.fc1(x10)
        x10 = self.output(x10)
        return x10, x1000

In [9]:
net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model = Model(net)
x10, x1000 = model(input_tensor)
print(x10.shape)
print(x1000.shape)

torch.Size([32, 10])
torch.Size([32, 1000])
